In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Load Dataset
df = pd.read_csv("../Data/The_Cancer_data_DP_Protected.csv")

# Split Features & Target Variable
X = df.drop(columns=["Diagnosis"]).values  # Convert to NumPy array
y = df["Diagnosis"].values

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for DP-SGD
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define Logistic Regression Model in PyTorch
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, 2)  # 2 outputs (Cancer Positive/Negative)
    
    def forward(self, x):
        return self.linear(x)

model = LogisticRegressionModel(input_size=X_train.shape[1])

# Define Loss Function & Optimizer (DP-SGD)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# Apply Differential Privacy Using Opacus
privacy_engine = PrivacyEngine()
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=10,
    target_epsilon=3.0,  # Privacy Budget
    target_delta=1e-5,
    max_grad_norm=1.0,  # Clipping Gradient Norm
)

# Train the DP-SGD Model
for epoch in range(10):
    for batch_idx, (features, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")

# Evaluate Model on Test Data
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor).argmax(dim=1).numpy()

# Calculate Accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\n DP-SGD Test Accuracy: {test_accuracy:.4f}")

# Save Model & Scaler
torch.save(model.state_dict(), "../dp_sgd_model.pth")
torch.save(scaler, "../scaler_SGD.pkl")
print("\n DP-SGD Model & Scaler Saved Successfully!")

c:\Users\ovaze\AppData\Local\Programs\Python\Python312\Lib\site-packages\opacus\privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
c:\Users\ovaze\AppData\Local\Programs\Python\Python312\Lib\site-packages\opacus\accountants\analysis\rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
c:\Users\ovaze\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using 

Epoch [1/10], Loss: 0.4622
Epoch [2/10], Loss: 0.5026
Epoch [3/10], Loss: 0.4172
Epoch [4/10], Loss: 0.3912
Epoch [5/10], Loss: 0.4927
Epoch [6/10], Loss: 0.3366
Epoch [7/10], Loss: 0.3545
Epoch [8/10], Loss: 0.5549
Epoch [9/10], Loss: 0.3957
Epoch [10/10], Loss: 0.3845

 DP-SGD Test Accuracy: 0.8167

 DP-SGD Model & Scaler Saved Successfully!
